# Computer Assignment 3

Due Date 12 ตุลาคม  2566 (ก่อนเวลา 23.00 น.)

นักศึกษาต้องสร้าง Classifier โดยเลือกจากที่เรียนในห้องเรียน หรือค้นคว้าจากวารสารทางวิชาการต่างๆ และนำมาทดลองกับ Ionosphere data set ซึ่ง data set ชุดนี้มีทั้งหมด 351 samples โดยแบ่งออกเป็น 2 class คือ good (g) class และ bad (b) class โดยที่ ใน g class มี 224 sample และใน b class มี 127 sample ใน data set ชุดนี้มี features ทั้งหมด 34 features ซึ่ง format ของ data set เป็น ดังนี้

    f1, f2,…,f34 Class

และนักศึกษาต้องทำการ select subset ของ feature ที่ดีที่สุด โดยเลือกใช้วิธีการที่เรียนในห้องเรียน หรือค้นคว้าจากวารสารทางวิชาการก็ได้

          อย่าลืมทำการประเมินประสิทธิภาพของวิธีการนี้ เช่นใช้ 10% cross validation

 

รายงานควรจะประกอบด้วย

          1. รายละเอียดของทฤษฎีหรือวิธีการต่างๆที่ใช้

          2. การออกแบบ algorithm เช่น pseudo-code, flowchart, ฯลฯ

          3. ผลการทดลอง

          4. การวิเคราะห์การทดลอง เช่น ได้ผลตามที่คาดไว้หรือไม่ มีสิ่งประหลาดเกิดขึ้นหรือไม่ บทสรุปที่ได้คืออะไร ฯลฯ

          5. Well documented, structured, modular program listing

          6. อย่าลืมแนบโปรแกรมมาในภาคผนวกด้วย

In [14]:
import pandas as pd

import pandas as pd

# open the txt file and read its contents
with open('ionosphere.txt', 'r', encoding='ISO-8859-1') as file:
    data = file.read()

# remove first 2 letters
data = data[2:]

# create a dataframe from the data
df = pd.DataFrame([row.split() for row in data.split('\n')])

# drop the row with odd index
df = df.drop(df.index[1::2])
# drop the last row
df = df.drop(df.index[-1])

# firt row is the column names
df.columns = df.iloc[0]
# drop the first row
df = df.drop(df.index[0])

# reset the index
df = df.reset_index(drop=True)
df



,f 1 , f 2 , f 3 , f 4 , f 5 , f 6 , f 7 , f 8 , f 9 , f 1 0 ,..., f 2 6 , f 2 7 , f 2 8 , f 2 9 , f 3 0 , f 3 1 , f 3 2 , f 3 3 , f 3 4 , c l a s s 
0, 1 , 0 , 0 . 9 9 5 3 9 , - 0 . 0 5 8 8 9 , 0 . 8 5 2 4 3 , 0 . 0 2 3 0 6 , 0 . 8 3 3 9 8 , - 0 . 3 7 7 0 8 , 1 , 0 . 0 3 7 6 ,..., - 0 . 5 1 1 7 1 , 0 . 4 1 0 7 8 , - 0 . 4 6 1 6 8 , 0 . 2 1 2 6 6 , - 0 . 3 4 0 9 , 0 . 4 2 2 6 7 , - 0 . 5 4 4 8 7 , 0 . 1 8 6 4 1 , - 0 . 4 5 3 , g 
1, 1 , 0 , 1 , - 0 . 1 8 8 2 9 , 0 . 9 3 0 3 5 , - 0 . 3 6 1 5 6 , - 0 . 1 0 8 6 8 , - 0 . 9 3 5 9 7 , 1 , - 0 . 0 4 5 4 9 ,..., - 0 . 2 6 5 6 9 , - 0 . 2 0 4 6 8 , - 0 . 1 8 4 0 1 , - 0 . 1 9 0 4 , - 0 . 1 1 5 9 3 , - 0 . 1 6 6 2 6 , - 0 . 0 6 2 8 8 , - 0 . 1 3 7 3 8 , - 0 . 0 2 4 4 7 , b 
2, 1 , 0 , 1 , - 0 . 0 3 3 6 5 , 1 , 0 . 0 0 4 8 5 , 1 , - 0 . 1 2 0 6 2 , 0 . 8 8 9 6 5 , 0 . 0 1 1 9 8 ,..., - 0 . 4 0 2 2 , 0 . 5 8 9 8 4 , - 0 . 2 2 1 4 5 , 0 . 4 3 1 , - 0 . 1 7 3 6 5 , 0 . 6 0 4 3 6 , - 0 . 2 4 1 8 , 0 . 5 6 0 4 5 , - 0 . 3 8 2 3 8 , g 
3, 1 , 0 , 1 , - 0 . 4 5 1 6 1 , 1 , 1 , 0 . 7 1 2 1 6 , - 1 , 0 , 0 ,..., 0 . 9 0 6 9 5 , 0 . 5 1 6 1 3 , 1 , 1 , - 0 . 2 0 0 9 9 , 0 . 2 5 6 8 2 , 1 , - 0 . 3 2 3 8 2 , 1 , b 
4, 1 , 0 , 1 , - 0 . 0 2 4 0 1 , 0 . 9 4 1 4 , 0 . 0 6 5 3 1 , 0 . 9 2 1 0 6 , - 0 . 2 3 2 5 5 , 0 . 7 7 1 5 2 , - 0 . 1 6 3 9 9 ,..., - 0 . 6 5 1 5 8 , 0 . 1 3 2 9 , - 0 . 5 3 2 0 6 , 0 . 0 2 4 3 1 , - 0 . 6 2 1 9 7 , - 0 . 0 5 7 0 7 , - 0 . 5 9 5 7 3 , - 0 . 0 4 6 0 8 , - 0 . 6 5 6 9 7 , g 
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346, 1 , 0 , 0 . 8 3 5 0 8 , 0 . 0 8 2 9 8 , 0 . 7 3 7 3 9 , - 0 . 1 4 7 0 6 , 0 . 8 4 3 4 9 , - 0 . 0 5 5 6 7 , 0 . 9 0 4 4 1 , - 0 . 0 4 6 2 2 ,..., - 0 . 0 4 2 0 2 , 0 . 8 3 4 7 9 , 0 . 0 0 1 2 3 , 1 , 0 . 1 2 8 1 5 , 0 . 8 6 6 6 , - 0 . 1 0 7 1 4 , 0 . 9 0 5 4 6 , - 0 . 0 4 3 0 7 , g 
347, 1 , 0 , 0 . 9 5 1 1 3 , 0 . 0 0 4 1 9 , 0 . 9 5 1 8 3 , - 0 . 0 2 7 2 3 , 0 . 9 3 4 3 8 , - 0 . 0 1 9 2 , 0 . 9 4 5 9 , 0 . 0 1 6 0 6 ,..., 0 . 0 1 3 6 1 , 0 . 9 3 5 2 2 , 0 . 0 4 9 2 5 , 0 . 9 3 1 5 9 , 0 . 0 8 1 6 8 , 0 . 9 4 0 6 6 , - 0 . 0 0 0 3 5 , 0 . 9 1 4 8 3 , 0 . 0 4 7 1 2 , g 
348, 1 , 0 , 0 . 9 4 7 0 1 , - 0 . 0 0 0 3 4 , 0 . 9 3 2 0 7 , - 0 . 0 3 2 2 7 , 0 . 9 5 1 7 7 , - 0 . 0 3 4 3 1 , 0 . 9 5 5 8 4 , 0 . 0 2 4 4 6 ,..., 0 . 0 3 1 9 3 , 0 . 9 2 4 8 9 , 0 . 0 2 5 4 2 , 0 . 9 2 1 2 , 0 . 0 2 2 4 2 , 0 . 9 2 4 5 9 , 0 . 0 0 4 4 2 , 0 . 9 2 6 9 7 , - 0 . 0 0 5 7 7 , g 
349, 1 , 0 , 0 . 9 0 6 0 8 , - 0 . 0 1 6 5 7 , 0 . 9 8 1 2 2 , - 0 . 0 1 9 8 9 , 0 . 9 5 6 9 1 , - 0 . 0 3 6 4 6 , 0 . 8 5 7 4 6 , 0 . 0 0 1 1 ,..., - 0 . 0 2 0 9 9 , 0 . 8 9 1 4 7 , - 0 . 0 7 7 6 , 0 . 8 2 9 8 3 , - 0 . 1 7 2 3 8 , 0 . 9 6 0 2 2 , - 0 . 0 3 7 5 7 , 0 . 8 7 4 0 3 , - 0 . 1 6 2 4 3 , g 


In [15]:
df.head()

,f 1 , f 2 , f 3 , f 4 , f 5 , f 6 , f 7 , f 8 , f 9 , f 1 0 ,..., f 2 6 , f 2 7 , f 2 8 , f 2 9 , f 3 0 , f 3 1 , f 3 2 , f 3 3 , f 3 4 , c l a s s 
0, 1 , 0 , 0 . 9 9 5 3 9 , - 0 . 0 5 8 8 9 , 0 . 8 5 2 4 3 , 0 . 0 2 3 0 6 , 0 . 8 3 3 9 8 , - 0 . 3 7 7 0 8 , 1 , 0 . 0 3 7 6 ,..., - 0 . 5 1 1 7 1 , 0 . 4 1 0 7 8 , - 0 . 4 6 1 6 8 , 0 . 2 1 2 6 6 , - 0 . 3 4 0 9 , 0 . 4 2 2 6 7 , - 0 . 5 4 4 8 7 , 0 . 1 8 6 4 1 , - 0 . 4 5 3 , g 
1, 1 , 0 , 1 , - 0 . 1 8 8 2 9 , 0 . 9 3 0 3 5 , - 0 . 3 6 1 5 6 , - 0 . 1 0 8 6 8 , - 0 . 9 3 5 9 7 , 1 , - 0 . 0 4 5 4 9 ,..., - 0 . 2 6 5 6 9 , - 0 . 2 0 4 6 8 , - 0 . 1 8 4 0 1 , - 0 . 1 9 0 4 , - 0 . 1 1 5 9 3 , - 0 . 1 6 6 2 6 , - 0 . 0 6 2 8 8 , - 0 . 1 3 7 3 8 , - 0 . 0 2 4 4 7 , b 
2, 1 , 0 , 1 , - 0 . 0 3 3 6 5 , 1 , 0 . 0 0 4 8 5 , 1 , - 0 . 1 2 0 6 2 , 0 . 8 8 9 6 5 , 0 . 0 1 1 9 8 ,..., - 0 . 4 0 2 2 , 0 . 5 8 9 8 4 , - 0 . 2 2 1 4 5 , 0 . 4 3 1 , - 0 . 1 7 3 6 5 , 0 . 6 0 4 3 6 , - 0 . 2 4 1 8 , 0 . 5 6 0 4 5 , - 0 . 3 8 2 3 8 , g 
3, 1 , 0 , 1 , - 0 . 4 5 1 6 1 , 1 , 1 , 0 . 7 1 2 1 6 , - 1 , 0 , 0 ,..., 0 . 9 0 6 9 5 , 0 . 5 1 6 1 3 , 1 , 1 , - 0 . 2 0 0 9 9 , 0 . 2 5 6 8 2 , 1 , - 0 . 3 2 3 8 2 , 1 , b 
4, 1 , 0 , 1 , - 0 . 0 2 4 0 1 , 0 . 9 4 1 4 , 0 . 0 6 5 3 1 , 0 . 9 2 1 0 6 , - 0 . 2 3 2 5 5 , 0 . 7 7 1 5 2 , - 0 . 1 6 3 9 9 ,..., - 0 . 6 5 1 5 8 , 0 . 1 3 2 9 , - 0 . 5 3 2 0 6 , 0 . 0 2 4 3 1 , - 0 . 6 2 1 9 7 , - 0 . 0 5 7 0 7 , - 0 . 5 9 5 7 3 , - 0 . 0 4 6 0 8 , - 0 . 6 5 6 9 7 , g 
